In [ ]:
print("Importing the modules...")
# Importing the required modules
import nltk
nltk.download('stopwords')
nltk.download('word_tokenize')
nltk.download('words')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import words
import pandas as pd
import re
# to check for valid english words
from nltk.corpus import words
# for the vector creation
import numpy as np
import spacy
import gensim
from gensim.models import Word2Vec
# to import the document
import io
# to detect the Mojibake strings
!pip install ftfy
import ftfy
from ftfy.badness import sequence_weirdness
# to detect the NaN (not a number) values in the csv
import math
nltk.download("punkt")
# Abstract Syntax Trees. A better alternative to the regex.
import ast
import time
# for a beep after running is over
# import winsound

frequency = 750  # 2500 Hz
duration = 500  # 1 sec

In [ ]:
# Trying Pydrive
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Main data file
link = "https://drive.google.com/open?id=1SMbOzDeQv8feCe8xnCCsXy210wund7V7"
# Sample data file
# link = "https://drive.google.com/open?id=1pViZwQKuD6WHP8BKfPCycZI2cFg5_Hi0"
fluff, id = link.split('=')
# print (id) # Verify that you have everything after '='

In [ ]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('zomato.csv')  
zomato = pd.read_csv('zomato.csv')
reviews = zomato['reviews_list']
# Dataset is now stored in a Pandas Dataframe

In [ ]:
# Input csv file
zomato = pd.read_csv('zomato.csv')
reviews = zomato['reviews_list']

In [ ]:
# # checking the output
for i in reviews[0:10]:
    print(i)
print(len(reviews))

In [ ]:
def processData(reviews):
    print("Diving the multiple reviews in the single cell...")
    # Dividing the multiple reviews in the single cell
    # individualReviews = []
    reviewBlocks = []
    # print(len(reviews))
    # print(math.isnan(reviews[0]))
    # reviews = reviews[0:6]
    # print(reviews)
    # reviewsFiltered = (i for i in reviews if i is not None)

    # --------------------------------------------
    # print(reviews[0])
    print("Removing the empty cells...")
    for i in reviews:
        # print(reviews)
        if i == "[]" or None:
            # print(reviews[i])
            # this is used when reviews is the whole data
            # reviews[i] = "garbage_value"
            # this is used when being sent in partition
            return None
            # reviews.remove(i)
            # print(reviews[i])
        try:
            if math.isnan(i):
            # print(reviews[i])
            # This is for collective input
                # reviews[i] = "garbage_value"
            # This is for single input
                return None
            # This is for 2 inputs
                # reviews.remove(i)
        except TypeError:
            continue
    print("Dividing the reviews...")

    for eachRestaurantReview in reviews:
        """1.1 eachRestaurant has all the reviews of one restaurant each """
        try:
            reviewBlocks = reviewBlocks + [list(x) if isinstance(x, tuple) else [x] for x in ast.literal_eval(eachRestaurantReview)]
        except ValueError:
            continue
    # --------------------------------------------------------
    filteredBlocksDivided = reviewBlocks

    # removing the keywords RATED and Rated
    for rBlock in filteredBlocksDivided:
        # print(rBlock)
        # This is because some of the reviews do no contain of the these. So, ignoring those.
        if rBlock[0] == None or rBlock[1] == None:
            filteredBlocksDivided.remove(rBlock)
            # Using continue will skip the rest of the iteration to start the next iteration
            continue
        rBlock[0] = rBlock[0].replace("Rated","")
        rBlock[0] = float(rBlock[0])
        # Removing only the first instance of the occurance of RATED in the review
        rBlock[1] = rBlock[1].replace("RATED","",1)
        rBlock[1] = rBlock[1].replace("\n", "").strip()
    
    # ---------------------------------------------------------
    print("Collecting the set of stop words...")
    # Collecting the set of stop words
    stopWords = set(stopwords.words('english'))
    """1.6 Filtering the review comments of any stop words. (Ignoring the 1.4)"""
    """1.7 Conducting extra filtering, removing the punctuations that's not covered in 1.6"""

    # The punctuationExtra is for the 1.7
    punctuationExtra = [".", ",", "''", "``", "(", ")", ":", ";", "{", "}", "[", "]", "!", "~", "`", "?", "'", "/"]
    numbers = ["0","1","2","3","4","5","6","7","8","9"]
    punctuationExtra = punctuationExtra + numbers
    # print(filteredBlocksDivided)
    # ---------------------------------------------------------------
    # To check the immediate duplicates. Ignoring the duplicates or else.
    reviewTokensOld = []
    # has all the review only in tokens
    reviewTokensFinal = []

    for review in filteredBlocksDivided:
        # Duplicate entries in the reviews. Getting rid of the them. "set" won't work as list is hashable - mutable (here)
        # print(review)
        # comment has the tokenized review[1] and review[0] is the rating
        # adding space after these to get good tokens
        review[1] = review[1].replace(".", " . ")
        review[1] = review[1].replace("'", " ' ")
        review[1] = review[1].replace("/", " ")
        # it's a waste to keep \\n still lying around
        review[1] = review[1].replace("\\n", "")
        reviewTokensNow = word_tokenize(review[1].lower())
        if reviewTokensNow == reviewTokensOld:
            # found a duplicate! Don't think have to remove as under because won't use it anyway
            filteredBlocksDivided.remove(review)
            continue
        else:
            # if not a duplicate then go on with the process
            # print(reviewTokens)
            # removing the stopwords
            # 1.6 being realised
            reviewTokensFiltered = [i for i in reviewTokensNow if i not in stopWords]
            # print(reviewTokensFiltered)
            # 1.7 being realised
            reviewTokensFurtherFiltered = [i for i in reviewTokensFiltered if i not in punctuationExtra]
            # print(reviewTokensFurtherFiltered)
            problemFlag = False
            # checking if these are corrupted i.e. valid english words - including variations - PROBLEM!!!
            for i in range(0, len(reviewTokensFurtherFiltered)):  # Most of them are variations - PROBLEM!!!
                # if reviewTokensFurtherFiltered[i] not in words.words():
                # Need to check the threshold later but for now putting it to be 0
                if sequence_weirdness(reviewTokensFurtherFiltered[i])> 0:
                    # found one garbage! Don't think have to remove as under because won't use it anyway
                    # filteredBlocksDivided.remove(review)
                    # print(reviewTokensFurtherFiltered[i])
                    problemFlag = True
                    break
                    # i = i+1
                # else:
                    # reviewTokensFinal = reviewTokensFinal + [reviewTokensFurtherFiltered]
            
            if not problemFlag:
            # tempSentence = " ".join([])
                reviewTokensFinal.append([review[0]," ".join(reviewTokensFurtherFiltered)])
        # updating to check later for duplicates
        reviewTokensOldOld = reviewTokensNow

    return reviewTokensFinal

In [ ]:
# USE THIS TO PROCESS THE ENTIRE DATA!!!

startTime = time.time()
reviewTokensFinal = []
print("The total number of restaurants to be processed = " + str(len(reviews)))
# This variable is to just keep track of the processing. No particular importance.
temp = 0
for i in reviews:
    print("Index = " + str(temp))
#     Arguments in the form of a list because the function is processing the lists not just the strings. 'i' is a string.
    processedReview = processData([i])
    if processedReview is not None:
        reviewTokensFinal = reviewTokensFinal + processedReview
    temp = temp + 1

print("All Done!")
print("--------%s seconds--------------" %(time.time()-startTime))
print("Number of reviews: " + str(len(reviewTokensFinal)))

In [ ]:
# Removing the duplicates
startTime = time.time()
reviewTokensFinal2 = []
for i in reviewTokensFinal:
    if i not in reviewTokensFinal2 and i not "Nan":
        reviewTokensFinal2.append(i)
        
print("----------%s seconds------------" %(time.time()-startTime))

In [ ]:
# There are empty reviews as well.
for i in reviewTokensFinal2:
    if i[1] == "Nan":
        print("ok")

In [ ]:
# Make a sound after done
# winsound.Beep(frequency, duration)
# Use this when using in Jupyter Notebooks
import csv
# reviewTokensFinal = list(set(reviewTokensFinal2))
# using utf-8 encoding here as there was an error regarding the encoding
with open('processedZomato_fullFinal.csv','w', newline = '', encoding = 'utf-8') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(reviewTokensFinal2)
csvFile.close()

In [ ]:
# Use this when using in Google colab
from google.colab import files
files.download('processedZomato_full.csv')

In [ ]:
print(len(reviewTokensFinal2))